In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

In [81]:
def freq_range(z, f_emit):
    if isinstance(z, list):
        f_obsv = []
        for redshift in z:
            f_obsv.append(round(f_emit / (1+redshift), 2))
    else:
        f_obsv = f_emit / (1+z)
    return f_obsv

In [85]:
freq_range(13, 1420)

101.42857142857143

In [83]:
z = [6, 7, 8, 10, 12, 13]

In [84]:
freq_range(z, 1420)

[202.86, 177.5, 157.78, 129.09, 109.23, 101.43]

In [5]:
X = np.array(([1,2], [3,4], [5,6], [7,8]))
Y = np.array((10,11,12,13))

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

In [15]:
X_train

array([[7, 8],
       [3, 4],
       [1, 2]])

In [16]:
X_test

array([[5, 6]])

In [18]:
X_train

array([[7, 8],
       [3, 4],
       [1, 2]])

In [19]:
X_test

array([[5, 6]])

In [61]:
a = np.arange(10)

In [62]:
np.random.seed(0)

In [65]:
np.random.shuffle(a)

In [60]:
a

array([9, 6, 8, 4, 5, 0, 2, 7, 3, 1])

In [66]:
a

array([9, 6, 8, 4, 5, 0, 2, 7, 3, 1])

In [25]:
%pwd

'/home/zoshs2/tf_gpu/Re'

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, ZeroPadding2D, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import argparse
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
from sklearn.model_selection import StratifiedShuffleSplit

In [32]:
X = Input((200,200,3))
X_input = ZeroPadding2D((100, 100))(X)
out = Conv2D(32, (6, 6), strides=(2, 2), padding='same')(X_input)
# out = MaxPooling2D((2,2), strides=3)(out)
model = Model(X, out)

In [33]:
model.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 200, 200, 3)]     0         
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 400, 400, 3)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 200, 200, 32)      3488      
Total params: 3,488
Trainable params: 3,488
Non-trainable params: 0
_________________________________________________________________


In [22]:
## CNN
print("Processing the training model")
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(200,200,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(32, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(64, (5,5),strides=(2,2),  padding='SAME')) # 패딩에 대한 결과는 strides에 따라 달라진다. padding을 "same"으로 해놓고 이미지사이즈를 변화시키고 싶지 않다면, strides((1,1)로 해야함)가 아닌 filters크기에 무관해진다.
                                                            # padding이 same 인 경우, output의 공간차원은 ceil(float(H_in)/float(Strides)), 반올림, 하여 계산된다. (즉, filter사이즈에 무관한 건 여전함)
                                                            # padding이 valid 인 경우, output의 공간차원은 ceil(float(H_in-Filter_size+1)/float(Strides)), 반올림, 하여 계산된다.( 이때는 filter 사이즈에 연관이 생긴다.)
                                                             
model.add(BatchNormalization())
model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Flatten())

## FC
model.add(Dense(64, kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(32, kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('linear'))

Processing the training model


In [23]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 200, 200, 32)      896       
_________________________________________________________________
batch_normalization_50 (Batc (None, 200, 200, 32)      128       
_________________________________________________________________
activation_60 (Activation)   (None, 200, 200, 32)      0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 200, 200, 32)      9248      
_________________________________________________________________
batch_normalization_51 (Batc (None, 200, 200, 32)      128       
_________________________________________________________________
activation_61 (Activation)   (None, 200, 200, 32)      0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 100, 100, 64)    